In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
import pandas as pd

crime_df = pd.read_csv('/content/drive/MyDrive/MADS Capstone Team 23/Data/raw/CrimeData/Dallas/Dallas-CrimeData-2014-2024.csv')

<ipython-input-31-36865f102378>:3: DtypeWarning: Columns (3,53,61) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_df = pd.read_csv('/content/drive/MyDrive/MADS Capstone Team 23/Data/raw/CrimeData/Dallas/Dallas-CrimeData-2014-2024.csv')


In [ ]:
crime_df.head(2)

,Incident Number w/year,Year of Incident,Service Number ID,Watch,Call (911) Problem,Type of Incident,Type Location,Type of Property,Incident Address,Apartment Number,...,NIBRS Code,NIBRS Group,NIBRS Type,Update Date,X Coordinate,Y Cordinate,Zip Code,City,State,Location1
0,203058-2022,2022,203058-2022-01,1,PSE/09 - THEFT,THEFT OF PROP (AUTO ACC) <$100 - (NOT EMP),Apartment Parking Lot,Motor Vehicle,7152 FAIR OAKS AVE,NaN,...,23G,A,Coded,2022-11-10 11:15:06.0000000,2.503598e+06,7.005279e+06,75231.0,DALLAS,TX,"7152 FAIR OAKS AVE\nDALLAS, TX 75231\n(32.8730..."
1,264509-2016,2016,264509-2016-01,2,58 - ROUTINE INVESTIGATION,FALSE STATEMENT FOR PROPERTY/CREDIT $2500 < $30K,Single Family Residence - Occupied,NaN,8545 MIDPARK RD,NaN,...,NaN,NaN,NaN,2017-03-31 00:43:58.0000000,2.503120e+06,7.028109e+06,75240.0,DALLAS,TX,"8545 MIDPARK RD\nDALLAS, TX 75240\n(32.93652, ..."


In [ ]:
all_type = crime_df["Type of Incident"].unique()

In [ ]:
print(*all_type, sep = "\n")

THEFT OF PROP (AUTO ACC) <$100 - (NOT EMP)
FALSE STATEMENT FOR PROPERTY/CREDIT $2500 < $30K
CREDIT CARD OR DEBIT CARD ABUSE
ABANDONED PROPERTY (NO OFFENSE)
THEFT OF PROP > OR EQUAL $100 <$750 (NOT SHOPLIFT) PC31.03(e2A)
ASSAULT -BODILY INJURY ONLY
FRAUD USE/POSS IDENTIFYING INFO-PRELIMINARY INVESTIGATION
CRIMINAL TRESPASS WARNING
CRIM MISCHIEF <$100
CRIM MISCHIEF > OR EQUAL $100 < $750
CRIM MISCHIEF > OR EQUAL $2,500 < $30K
BURGLARY OF BUILDING - FORCED ENTRY
UNAUTHORIZED USE OF MOTOR VEH - AUTOMOBILE
BMV
DEADLY CONDUCT DISCHARGE FIREARM
HARASSMENT
CRIM MISCHIEF > OR EQUAL $1,500 BUT < $20K
CREDIT CARD OR DEBIT CARD ABUSE - PRELIMINARY INVESTIGATION
ACCIDENTAL DEATH (NO OFFENSE)
THEFT OF PROP > OR EQUAL $750 <$2,500 (NOT SHOPLIFT) PC31.03(e3)
CRUELTY TO NON-LIVESTOCK ANIMALS: FAILURE TO PROVIDE FOR
INJURED PERSON - AT HOME (OTHER THAN FIREARM) (NO OFFENSE)
UNLAWFUL CARRYING WEAPON
ASSAULT (AGG) -DEADLY WEAPON
THEFT OF PROP > OR EQUAL $100 <$750 (SHOPLIFT-NOT EMP) PC31.03(e2A)
FOUND PRO

In [32]:
violent_keywords = ["ASSAULT", "ROBBERY", "MURDER", "KIDNAPPING", "DEADLY CONDUCT", "TERRORISTIC THREAT", "STALKING", "ARSON", "ATTACK", "INJURY"]
property_keywords = ["THEFT", "BURGLARY", "CRIMINAL TRESPASS", "FRAUD", "MISCHIEF", "VANDALISM", "GRAFFITI", "TRESPASS", "POSSESSION OF STOLEN PROPERTY", "FORGERY"]

def categorize_incident(incident):
    incident = str(incident)
    for keyword in violent_keywords:
        if keyword in incident:
            return "violent_crimes"
    for keyword in property_keywords:
        if keyword in incident:
            return "property_crimes"
    return "unknown"

crime_df['Category'] = crime_df['Type of Incident'].apply(categorize_incident)

# Filter the DataFrame into violent and property crimes
violent_crime_df = crime_df[crime_df['Category'] == "violent_crimes"]
property_crime_df = crime_df[crime_df['Category'] == "property_crimes"]

In [33]:
violent_crime_df["Date of Report"] = pd.to_datetime(violent_crime_df["Date of Report"])
property_crime_df["Date of Report"] = pd.to_datetime(property_crime_df["Date of Report"])

<ipython-input-33-fe0df552c1be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  violent_crime_df["Date of Report"] = pd.to_datetime(violent_crime_df["Date of Report"])
<ipython-input-33-fe0df552c1be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_crime_df["Date of Report"] = pd.to_datetime(property_crime_df["Date of Report"])


In [34]:
violent_crimes_by_month = violent_crime_df.groupby(pd.Grouper(key='Date of Report', freq='M')).size().reset_index(name='NumberOfIncident')
violent_crimes_by_month['Date of Report'] = violent_crimes_by_month['Date of Report'].dt.strftime('%Y-%m')

property_crimes_by_month = property_crime_df.groupby(pd.Grouper(key='Date of Report', freq='M')).size().reset_index(name='NumberOfIncident')
property_crimes_by_month['Date of Report'] = property_crimes_by_month['Date of Report'].dt.strftime('%Y-%m')

In [35]:
violent_crimes_by_month.rename(columns={"Date of Report": "Date"}, inplace=True)
property_crimes_by_month.rename(columns={"Date of Report": "Date"}, inplace=True)


In [12]:
violent_crime_2010 = 148+505+4487+4021
property_crime_2010 = 19594+36147+8384+642

violent_crime_2011 = 133+428+4066+3703
property_crime_2011 = 18727+35148+7984+596

violent_crime_2012 = 154+486+4093+3647
property_crime_2012 = 16090+31148+7062+581

violent_crime_2013 = 143+543+4202+3442
property_crime_2013 = 14516+30374+7384+455

violent_crime_2014 = 116+781+3856+3704
property_crime_2014 = 11685+26939+7045+385

In [13]:
# Create a dictionary with the data
data = {
    'Year': [2010, 2011, 2012, 2013, 2014],
    'ViolentCrimeIncident': [violent_crime_2010, violent_crime_2011, violent_crime_2012, violent_crime_2013, violent_crime_2014],
    'PropertyCrimeIncident': [property_crime_2010, property_crime_2011, property_crime_2012, property_crime_2013, property_crime_2014]
}

# Create the DataFrame
yearly_df = pd.DataFrame(data)
yearly_df['City'] = 'Dallas'
# Display the DataFrame
print(yearly_df)


   Year  ViolentCrimeIncident  PropertyCrimeIncident    City
0  2010                  9161                  64767  Dallas
1  2011                  8330                  62455  Dallas
2  2012                  8380                  54881  Dallas
3  2013                  8330                  52729  Dallas
4  2014                  8457                  46054  Dallas


In [25]:
# Extract the monthly data for Dallas Crime
property_crime_df = pd.read_csv(f'/content/drive/MyDrive/MADS Capstone Team 23/Data/processed/Crime/dallas_property_crimes_by_month.csv')
violent_crime_df = pd.read_csv(f'/content/drive/MyDrive/MADS Capstone Team 23/Data/processed/Crime/dallas_violent_crimes_by_month.csv')

# Rename columns
property_crime_df.rename(columns={"NumberOfIncident": "PropertyCrimeIncident"}, inplace=True)
violent_crime_df.rename(columns={"NumberOfIncident": "ViolentCrimeIncident"}, inplace=True)

# Merge dataframes
monthly_df = pd.merge(property_crime_df, violent_crime_df, on='Date')

# Add city column
monthly_df['City'] = 'Dallas'

# Extract data from 2015-2023
monthly_df = monthly_df[(monthly_df['Date']>'2014-12') & (monthly_df['Date'] <'2024-01')]

monthly_df['Date'] = pd.to_datetime(monthly_df['Date'])

monthly_df['Year'] = monthly_df['Date'].dt.year
monthly_df['Month'] = monthly_df['Date'].dt.month

yearly_totals = monthly_df.groupby('Year').agg({
    'PropertyCrimeIncident': 'sum',
    'ViolentCrimeIncident': 'sum'
}).reset_index()

monthly_df = monthly_df.merge(yearly_totals, on='Year', suffixes=('', '_YearlyTotal'))

monthly_df['PropertyCrimeProportion'] = monthly_df['PropertyCrimeIncident'] / monthly_df['PropertyCrimeIncident_YearlyTotal']
monthly_df['ViolentCrimeProportion'] = monthly_df['ViolentCrimeIncident'] / monthly_df['ViolentCrimeIncident_YearlyTotal']

monthly_proportions = monthly_df.groupby('Month')[['PropertyCrimeProportion', 'ViolentCrimeProportion']].mean().reset_index()

monthly_proportions

,Month,PropertyCrimeProportion,ViolentCrimeProportion
0,1,0.083696,0.077577
1,2,0.071787,0.065901
2,3,0.081268,0.077401
3,4,0.081196,0.079661
4,5,0.083028,0.089489
5,6,0.083683,0.088745
6,7,0.089457,0.093603
7,8,0.089469,0.091097
8,9,0.084611,0.085831
9,10,0.085567,0.087869


In [30]:
def distribute_yearly_to_monthly(yearly_df, proportions_df):
    distributed_data = []

    for _, row in yearly_df.iterrows():
        year = row['Year']
        for _, prop_row in proportions_df.iterrows():
            month = int(prop_row['Month'])  # Ensure month is an integer
            property_crime = row['PropertyCrimeIncident'] * prop_row['PropertyCrimeProportion']
            violent_crime = row['ViolentCrimeIncident'] * prop_row['ViolentCrimeProportion']
            distributed_data.append([f'{year}-{month:02d}', property_crime, violent_crime, row['City']])

    distributed_df = pd.DataFrame(distributed_data, columns=['Date', 'PropertyCrimeIncident', 'ViolentCrimeIncident', 'City'])
    return distributed_df

# Distribute yearly data to monthly
distributed_monthly_df = distribute_yearly_to_monthly(yearly_df, monthly_proportions)

# Convert Date to datetime
distributed_monthly_df['Date'] = pd.to_datetime(distributed_monthly_df['Date'])

distributed_monthly_df['ViolentCrimeIncident'] = distributed_monthly_df['ViolentCrimeIncident'].round().astype(int)
distributed_monthly_df['PropertyCrimeIncident'] = distributed_monthly_df['PropertyCrimeIncident'].round().astype(int)

In [44]:
violent_2010_2014 = distributed_monthly_df[['Date', 'ViolentCrimeIncident']]
violent_2010_2014['Date'] = violent_2010_2014['Date'].dt.strftime('%Y-%m')
violent_2010_2014.rename(columns={"ViolentCrimeIncident": "NumberOfIncident"}, inplace=True)

property_2010_2014 = distributed_monthly_df[['Date', 'PropertyCrimeIncident']]
property_2010_2014['Date'] = property_2010_2014['Date'].dt.strftime('%Y-%m')
property_2010_2014.rename(columns={"PropertyCrimeIncident": "NumberOfIncident"}, inplace=True)

<ipython-input-44-e39950be2a9e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  violent_2010_2014['Date'] = violent_2010_2014['Date'].dt.strftime('%Y-%m')
<ipython-input-44-e39950be2a9e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  violent_2010_2014.rename(columns={"ViolentCrimeIncident": "NumberOfIncident"}, inplace=True)
<ipython-input-44-e39950be2a9e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [45]:
violent_2015_2024 = violent_crimes_by_month[(violent_crimes_by_month['Date']>'2014-12')]
property_2015_2024 = property_crimes_by_month[(property_crimes_by_month['Date']>'2014-12')]

,Date,NumberOfIncident
150,2015-01,659
151,2015-02,538
152,2015-03,666
153,2015-04,701
154,2015-05,787
...,...,...
260,2024-03,946
261,2024-04,1062
262,2024-05,1054
263,2024-06,1089


In [46]:
violent_all = pd.concat([violent_2010_2014, violent_2015_2024])
property_all = pd.concat([property_2010_2014, property_2015_2024])


In [48]:
violent_all.to_csv("/content/drive/MyDrive/MADS Capstone Team 23/Data/processed/Crime/dallas_violent_crimes_by_month.csv", index=False)
property_all.to_csv("/content/drive/MyDrive/MADS Capstone Team 23/Data/processed/Crime/dallas_property_crimes_by_month.csv", index=False)